In [43]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import math 


In [44]:
df = pd.read_csv("sample_data\\201907-citibike-tripdata.csv")


In [45]:
df2 = df[["start station id", "start station name", "start station latitude", "start station longitude"]]
df = df[["end station id", "end station name", "end station latitude", "end station longitude"]]


df = df.groupby(["end station id", "end station name"], group_keys=False).mean()
df2 = df2.groupby(["start station id", "start station name"], group_keys=False).mean()

In [46]:
df

,,end station latitude,end station longitude
end station id,end station name,,
72.0,W 52 St & 11 Ave,40.767272,-73.993929
79.0,Franklin St & W Broadway,40.719116,-74.006667
82.0,St James Pl & Pearl St,40.711174,-74.000165
83.0,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323
116.0,W 17 St & 8 Ave,40.741776,-74.001497
...,...,...,...
3791.0,Hoboken Ave at Monmouth St,40.735208,-74.046964
3793.0,Stuyvesant Walk & 1 Av Loop,40.732340,-73.978984
3795.0,10 St & 2 Ave,40.671907,-73.993612


In [47]:
df["end station latitude"] = round(df["end station latitude"], 4)
df["end station longitude"] = round(df["end station longitude"], 4)
df2["start station latitude"] = round(df2["start station latitude"], 4)
df2["start station longitude"] = round(df2["start station longitude"], 4)

In [48]:
df.reset_index(inplace=True)
df
df2.reset_index(inplace=True)
df2.columns = df.columns
df = df.append(df2)
df = df.drop_duplicates(["end station id"])
# df = df.drop(["index"], axis=1)
df


,end station id,end station name,end station latitude,end station longitude
0,72.0,W 52 St & 11 Ave,40.7673,-73.9939
1,79.0,Franklin St & W Broadway,40.7191,-74.0067
2,82.0,St James Pl & Pearl St,40.7112,-74.0002
3,83.0,Atlantic Ave & Fort Greene Pl,40.6838,-73.9763
4,116.0,W 17 St & 8 Ave,40.7418,-74.0015
...,...,...,...,...
795,3791.0,Hoboken Ave at Monmouth St,40.7352,-74.0470
796,3793.0,Stuyvesant Walk & 1 Av Loop,40.7323,-73.9790
797,3795.0,10 St & 2 Ave,40.6719,-73.9936
798,3798.0,W 40 St & 5 Ave,40.7523,-73.9821


In [49]:
df

,end station id,end station name,end station latitude,end station longitude
0,72.0,W 52 St & 11 Ave,40.7673,-73.9939
1,79.0,Franklin St & W Broadway,40.7191,-74.0067
2,82.0,St James Pl & Pearl St,40.7112,-74.0002
3,83.0,Atlantic Ave & Fort Greene Pl,40.6838,-73.9763
4,116.0,W 17 St & 8 Ave,40.7418,-74.0015
...,...,...,...,...
795,3791.0,Hoboken Ave at Monmouth St,40.7352,-74.0470
796,3793.0,Stuyvesant Walk & 1 Av Loop,40.7323,-73.9790
797,3795.0,10 St & 2 Ave,40.6719,-73.9936
798,3798.0,W 40 St & 5 Ave,40.7523,-73.9821


In [6]:
geolocator = Nominatim(user_agent="bikes")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=0)

list_of_addresses  = df.apply(lambda row: reverse(str(row["end station latitude"]) + ", " + str(row["end station longitude"])).raw["address"], axis=1)


In [7]:
addresses = pd.DataFrame(list(list_of_addresses))
addresses

,amenity,road,neighbourhood,suburb,county,city,state,ISO3166-2-lvl4,postcode,country,...,highway,man_made,leisure,historic,office,tourism,industrial,town,emergency,place
0,Citi Bike - W 52 St / 11 Ave,West 52nd Street,Hell's Kitchen,Manhattan,New York County,City of New York,New York,US-NY,10019,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Citi Bike - Franklin St & W Broadway,Franklin Street,Tribeca,NaN,New York County,City of New York,New York,US-NY,10005,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Citi Bike - Saint James Place & Pearl Street,Pearl Street,Two Bridges,Manhattan,New York County,City of New York,New York,US-NY,10038,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Citi Bike - Atlantic Avenue & Fort Greene Place,Atlantic Avenue,NaN,Brooklyn,NaN,City of New York,New York,US-NY,11208,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Citi Bike - W 17 St & 8 Ave,West 17th Street,Chelsea District,NaN,New York County,City of New York,New York,US-NY,10011,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,NaN,Hoboken Avenue,NaN,NaN,Hudson County,Jersey City,New Jersey,US-NJ,07310,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
796,NaN,Stuyvesant Walk,NaN,NaN,New York County,City of New York,New York,US-NY,10009,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
797,NaN,10th Street,Park Slope,Brooklyn,NaN,City of New York,New York,US-NY,11215,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
798,NaN,West 40th Street,Manhattan Community Board 5,NaN,New York County,City of New York,New York,US-NY,10018,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df[["Suburb", "Road", "Neighborhood"]] = addresses[["suburb", "road", "neighbourhood"]]

In [9]:
df

,end station id,end station name,end station latitude,end station longitude,Suburb,Road,Neighborhood
0,72.0,W 52 St & 11 Ave,40.7673,-73.9939,Manhattan,West 52nd Street,Hell's Kitchen
1,79.0,Franklin St & W Broadway,40.7191,-74.0067,NaN,Franklin Street,Tribeca
2,82.0,St James Pl & Pearl St,40.7112,-74.0002,Manhattan,Pearl Street,Two Bridges
3,83.0,Atlantic Ave & Fort Greene Pl,40.6838,-73.9763,Brooklyn,Atlantic Avenue,NaN
4,116.0,W 17 St & 8 Ave,40.7418,-74.0015,NaN,West 17th Street,Chelsea District
...,...,...,...,...,...,...,...
795,3791.0,Hoboken Ave at Monmouth St,40.7352,-74.0470,NaN,Hoboken Avenue,NaN
796,3793.0,Stuyvesant Walk & 1 Av Loop,40.7323,-73.9790,NaN,Stuyvesant Walk,NaN
797,3795.0,10 St & 2 Ave,40.6719,-73.9936,Brooklyn,10th Street,Park Slope
798,3798.0,W 40 St & 5 Ave,40.7523,-73.9821,NaN,West 40th Street,Manhattan Community Board 5


In [10]:
df.columns = ["StationID", "StationName", "Latitude", "Longitude", "Suburb", "Road", "Neighborhood"]


In [11]:
df.to_csv("stations2.csv")